In [ ]:
import arcpy
import os
import sys
ws = r'C:\Users\Ricardo\Desktop\PCC\Internship\UpdatedCountyTaxLots\TaxLots\TaxLots.gdb'
arcpy.env.workspace = ws

In [ ]:
def unique_values(table, field):
    with arcpy.da.SearchCursor(table, [field]) as cursor:
    
        return sorted({row[0] for row in cursor if row[0] != None})
        del cursor

In [ ]:
def sqlClause(termList, fc, fieldName):
    owner_string = ""
    for owner in termList:
        if termList.index(owner) != len(termList) -1:
            owner_string = owner_string + """{field} = '{owner}' Or """.format(field = arcpy.AddFieldDelimiters(fc, fieldName), 
                                                                               owner = owner)
        elif termList.index(owner) == len(termList) -1:
            owner_string = owner_string + """{field} = '{owner}'""".format(field = arcpy.AddFieldDelimiters(fc, fieldName), 
                                                                               owner = owner)
    return owner_string

In [ ]:
def removeTerms(OgList, removeList):
    for item in removeList:
        if item in OgList:
            OgList.remove(item)
    return OgList
    

In [ ]:
def narrowList(ogList):
    ogList = [i for i in ogList if not ('ESTATE' in i or 'SUSAN' in i or 'LLC' in i or 
                                                'COMPANY' in i or 'ASSOC' in i or 'ASSN' in i or
                                        'INC' in i or 'BANK' in i or 'CHURCH' in i or 'SOCIETY' in i or
                                               'RAILWAY' in i or 'PRESBYTERIAN' in i or 'CORP' in i or
                                       'FUND' in i)]
    
    removeList = [i for i in ogList if ('US' in i and not i.startswith('US'))]
    
    ogList = removeTerms(ogList, removeList)
    
    return ogList

In [ ]:
def pubFind(myValues, searchTerms = ['USA', 'UNITED', 'STATE', 'OREGON', 'WASH', 'COUNTY', 'SKAMANIA', 'CITY', 'CDID', 'GOVERNMENT', 'SCHOOL', 'PUBLIC', 'PORT OF', 'METRO', 'DISTRICT']):
    pubLand = []
    myValues = [value.upper() for value in myValues]
    
    for owner in myValues:
        for term in searchTerms:
            if term in owner:
                pubLand.append(owner)
    
    
   
    return pubLand

In [ ]:
# Performed repair geometry onold parcels merge in ArcGis


In [ ]:
# Creat point feature class from old_parcels_merge
arcpy.FeatureToPoint_management('old_parcels_merge', 'oldParcelsPoints', 'INSIDE')

In [ ]:
# Select only the points that are privately owned from that select points that intersect 
# with publicLands layer and spatial join with public lands to get owner information
pointsIntersect = arcpy.SelectLayerByLocation_management('oldParcelsPoints', 'WITHIN', 'publicLandsFloodplain')
privatePoints = arcpy.SelectLayerByAttribute_management(pointsIntersect, 'SUBSET_SELECTION', where_clause = """PUB_PVT = 'PRIVATE'""")
arcpy.CopyFeatures_management(privatePoints, 'privatePoints')

In [ ]:
# Display unique owners in newly created layer
owners = unique_values('privatePoints', 'Land_Owner')

In [ ]:
pubOwners = pubFind(owners)
pubOwners = narrowList(pubOwners)

In [ ]:
pubOwners = [i for i in pubOwners if not ('LEASED' in i)]

In [ ]:
pubOwners

In [ ]:
privOwners = removeTerms(owners, pubOwners)

In [ ]:
badString = pubFind(privOwners, searchTerms = "'")

In [ ]:
# Remove ' form any owners
with arcpy.da.UpdateCursor('privatePoints', 'Land_Owner') as cursor:
    for row in cursor:
        if row[0] in badString:
           
            row[0] = row[0].replace("'", "")
        cursor.updateRow(row)
    del cursor
            

In [ ]:
owners = unique_values('privatePoints', 'Land_Owner')
pubOwners = pubFind(owners)
pubOwners = narrowList(pubOwners)
pubOwners = [i for i in pubOwners if not ('LEASED' in i)]
privOwners = removeTerms(owners, pubOwners)
badString = pubFind(privOwners, searchTerms = "'")
print(badString)

In [ ]:
# Refine private owners list further
removePublicLands = ['COLUMBIA 9-1-1 COMMUNICATIONS DISTRICT', 'COLUMBIA HEALTH DISTRICT', 'COWLITZ VALLEY POST #1045',
                    'HAZEL DELL SEWER DISTRICT', 'HOUSING AUTHORITY VANCOUVER', 'METRO(LEASED', 'METRO> & PORTLAND CITY OF>',
                    'MULTNOMAH COUNTY TAX TITLE', 'MULTNOMAH COUNTY(LEASED', 'MULTNOMAH COUNTY(PERMIT', 'OREGON STATE OF (LEASED', 
                     'OREGON STATE OF(DIV OF STATE L', 'OREGON STATE OF(DVA', 'OREGON STATE OF(EASEMENT', 'OREGON STATE OF(ESMNT PGE & PO', 
                     'OREGON STATE OF(LEASED', 'OREGON STATE OF(LEASED AK MEDI', 'OREGON STATE OF(LEASED BELMONT', 
                     'OREGON STATE OF(LEASED GRAND &', 'OREGON STATE OF(LEASED PENSKE', 'OREGON STATE OF(LEASED PORTLAN', 
                     'OREGON STATE OF(LEASED RED LIO', 'OREGON STATE OF(LEASED SEQUOIA', 'OREGON STATE OF(LEASED SOLHEIM', 
                     'OREGON STATE OF(LEASED WINMAR', 'OREGON STATE(LEASED', 'PORT OF VANCOUVER CONT', 'PUBLIC', 
                     'PUBLIC HEALTH CARE SERVICES #3 OF PACI', 'PUD #1', 'PUBLIC', 'PUBLIC HEALTH CARE SERVICES #3 OF PACI', 'PUD #1',
                      'U S A', 'UNITED STATES POSTAL SERVICE', 'UNITED STATES POSTAL SERVICE>', 'UNIVERSITY OF PORTLAND',
                    'US NATURAL RESOURCES', 'WASH STATE DEPT OF WILDLIFE', 'WASHINGTON DEPT OF WILDLIFE']

In [ ]:
privOwners = removeTerms(privOwners, removePublicLands)

In [ ]:
print(privOwners)

In [ ]:
sql = sqlClause(privOwners, 'privatePoints', 'Land_Owner')

In [ ]:
# CReate Selection of public lands that contain private points
# and spatial join together to get polygons that have been made public since last update
publicSinceLastUpdate = arcpy.SelectLayerByAttribute_management('privatePoints', where_clause = sql)
subSet = arcpy.SelectLayerByLocation_management('publicLandsFloodplain', 'COMPLETELY_CONTAINS', publicSinceLastUpdate)
arcpy.SpatialJoin_analysis(subSet, 'privatePoints', 'publicizedLands', match_option = 'COMPLETELY_CONTAINS')